# Ejercicio 1 historico de RV de cada activo de muchos indices Yahoo

- Obtén el histórico (60 días) de cada activo de varios índices (usa el código del día anterior)
- Recomendación: Yahoo Finance . # Tiempo objetivo: 45 minutos
- La clave está en qué tenemos que hacer cuando un activo de error en la descarga...

En este ejercicio vamos incluir los siguientes controles de error:

      Control 1: En la conexión a la web haremos 3 intentos, y si no lo conseguimos, seguimos
      Control 2: En los errores de descarga que habiamos ya esbozado en ejercicios previos, hay que devolver la causa del error

In [13]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime
import time
from tqdm import tqdm
import math
from rcurl import get_curl
from io import BytesIO
from time import mktime
from datetime import timedelta

Funcion que recupera el los datos en bruto de Yahoo finance, realizando el chequeo del crumb.
Ver https://maikros.github.io/yahoo-finance-python/

In [7]:
def _get_crumbs_and_cookies(stock):
    """
    get crumb and cookies for historical data csv download from yahoo finance  
    parameters: stock - short-handle identifier of the company    
    returns a tuple of header, crumb and cookie
    """   
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    
    with requests.session():
        
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }  
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        
        try:
            crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))
            output=(header, crumb[0], website.cookies)
            return output
        
        except:
            
            print('no lo podemos descargar')

Establecemos el rango de fechas que queremos importar.
Yahoo utiliza formato de fecha poxis - time unix
(cantidad de segundos transcurridos desde la medianoche UTC del 1 de enero de 1970, teniendo cada día 86400 segundos)
Convertimos la fecha a formato poxis

In [8]:
def convert_to_unix(date):
    """
    converts date to unix timestamp
    parameters: date - in format (yyyy-mm-dd)
    returns integer unix timestamp
    """
    datum = datetime.strptime(date, '%Y-%m-%d')
    
    return int(mktime(datum.timetuple()))

In [9]:
def load_csv_data(stock, interval='1d', day_begin='20-03-2018', day_end='20-06-2018'):
    """
    queries yahoo finance api to receive historical data in csv file format
    
    parameters: 
        stock - short-handle identifier of the company
        interval - 1d, 1wk, 1mo - daily, weekly monthly data
        day_begin - starting date for the historical data (format: dd-mm-yyyy)
        day_end - final date of the data (format: dd-mm-yyyy)
    
    returns a list of comma seperated value lines
    """
    
    error1='404 Not Found: Timestamp data missing.'
    
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
       
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
       
        return website.text.split('\n')

In [10]:
def prueba_conexion1(max_intentos,error1):
    
    intentos=1
    valor_check=website.text.split('\n')
    
    for i in range(max_intentos):
        
        if intentos==3:
            print('no intentamos mas')
            break
            
        if valor_check==error1 & max_intentos<=3:
            intentos=intentos+1
            Sys.sleep(5)
            print('reintentamos',intentos)
            website = requests.get(url, headers=header, cookies=cookies)
            valor_check=website.text.split('\n') 
            
        else:
            return valor_check

In [11]:
def prueba_conexion2(max_intentos,error1=''):
    
    intentos=1
    valor_check=website.text.split('\n')
    
    for i in range(max_intentos):
        
        if intentos==3:
            print('no intentamos mas')
            break
            
        if valor_check==error1 & max_intentos<=3:
            intentos=intentos+1
            Sys.sleep(5)
            print('reintentamos',intentos)
            website = requests.get(url, headers=header, cookies=cookies)
            valor_check=website.text.split('\n')  
            
        else:
            return valor_check

In [12]:
def componentes_indice(tickers_de_indices):
    """esta funcion devuelve los componentes y divisa de cada indice.
    como parametro se debe inclur los tickers de Yahoo Finance de los indices
    en una tupla"""

    n=0
    for i in tickers_de_indices:

        Index=i
        
        #primero se obtiene la divisa
        url="https://es.finance.yahoo.com/quote/"+i+"/components/"
        soup  = requests.get(url)
        soup  = BeautifulSoup(soup.content, 'html.parser')       
        name_box = soup.find('span', attrs={'data-reactid': '4'})
        
        try:
            name = name_box.text.strip()
            divisa=name[len(name)-3:len(name)]
            print(i,divisa)
            
            # Reconstruimos la tabla. Tiene 6 columnas (símbolo, nombre de la empresa, último precio, cambio, cambio % y volumen.)
            # Nos interesa obtener el símbolo y el nombre de la empresa.
            # Ojo, queremos importar únicamente los activos que tengan cotización. 
            
            soup  = requests.get(url)
            soup  = BeautifulSoup(soup.content, 'html.parser')
           
            name_box = soup.find_all('tr')
            name_boxccc=name_box[1]
            name_boxccc = name_boxccc.get_text(strip=True)
            lista_activo2=[]
            lista_indice=[]
            lista_divisa=[]
            lista_num_errores=[]
            
            lista_resto=[]

            for i in range(1,len(name_box)-1):
                
                try:
                    lista_indice.append(Index)
                    name_boxccc=name_box[i]
                    name_boxccc = name_boxccc.get_text(strip=True)
                    name_boxccc=name_boxccc.split(sep='.')
                    
                    if name_boxccc[0]=='656690656690': #este caso es un activo  que me rompe la descarga del índice brasileno (mejorar)
                        
                        lista_activo2.append(name_boxccc[0])
                        lista_resto.append(name_boxccc)
                        lista_divisa.append(divisa)
                        
                    else:
                        lista_activo2.append(name_boxccc[0]+'.'+name_boxccc[1][0:2])                        
                        lista_resto.append(name_boxccc)
                        lista_divisa.append(divisa)

                except:
                    lista_num_errores.append(i)

            if len(lista_activo2)==len(lista_divisa):

                df=pd.DataFrame ({'indice':lista_indice,
                                'activo':lista_activo2,
                                'divisa':lista_divisa,
                                'resto':lista_resto})
                
                df['posicion']=np.where(df['resto'].astype(str).str.contains('%', regex=False)==True,1,0)

                df=df[df['posicion']==1]
                df=df.drop(['resto','posicion'],1)

                if n==0:
                    dfacum=df
                else:
                    dfacum=pd.concat([dfacum, df], axis=0,sort=True)
                    dfacum.reset_index(drop=True)
                    
                n=n+1

            else:
                print('El activo '+str(i)+'no se ha podido incorporar por inconsistencias en los datos importados')

        except:
            print('Para la referencia '+str(i)+' no se ha podido descargar la informacion')
    
    return dfacum    

In [14]:
def set_dates(x,ventana):
    """setup inicial de fechas,
    esta funcion devuelve la fecha de inicio
    y fin en string y formato YYYY-MM-DD
    Parametros:
    x=start o end
    ventana: entero referido al numero de dias del periodo"""
    
    hoy=datetime.now()
    
    if x=='end':
        fecha_fin = str(hoy.now())
        fecha_fin = fecha_fin[0:10]
        return (fecha_fin)
    
    if x=='start':
        fecha_inicial=hoy-timedelta(ventana)
        fecha_inicial = str(fecha_inicial)
        fecha_inicial = fecha_inicial[0:10]
        return (fecha_inicial)
    
    else:
        print('inputs incorrectos. Ver docstring d ela funcion')       

In [15]:
def historic_prices_series(stock_series,interval,fecha_inicial,fecha_fin):
    
    n=0
    x=0 
    
    for benchmark in stock_series:
        
        print("descargando "+benchmark+' del indice')
        
        try:
            DF=load_csv_data(benchmark, interval=interval, day_begin=fecha_inicial, day_end=fecha_fin)
            DF=pd.DataFrame(DF)
            DF = DF.iloc[:,0].str.split(",", n = 7, expand = True)
            DF.columns=DF.iloc[0,:]
            DF=DF.iloc[1:DF.shape[0],:]
            x=DF.shape[1]  
            
            if x<2:
                 print("No nos hemos podido descargar el activo", benchmark)
                    
            else:
                DF['stock']=str(benchmark)
                DF=DF.dropna()
            
                if n==0:
                    dfacum=DF
                else:
                    dfacum=agrega_dataframes(DF,dfacum)    
                n=n+1
                
        except KeyError as e:
            
            print("No nos hemos podido descargar el indice")
            print(f'key error: {e}')
        
    return dfacum

In [16]:
def agrega_dataframes(df,dfacum):
    """codigo que permite ir concatenando
    dataframes con estructura similar
    df:daframe a agregar
    dfacum: dataframe en donde se agregara"""
    
    dfacum=pd.concat([dfacum, df], axis=0,sort=True)
    dfacum.reset_index(drop=True)
    return dfacum

#### Ejecucion

In [28]:
tickers_de_indices=("%5EBFX","%5EBVSP","%5EFCHI", "%5EGDAXI", "%5EHSI", "%5EIBEX") 
                    #"%5EMXX", "%5EJKSE", "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI")

ventana=60

In [18]:
dfacum = componentes_indice(tickers_de_indices)

%5EBFX EUR
%5EBVSP BRL
%5EFCHI EUR
%5EGDAXI EUR
%5EHSI HKD
%5EIBEX EUR
%5EMXX MXN
Para la referencia %5EJKSE no se ha podido descargar la informacion
%5EMERV  en
%5EOMXSPI SEK
%5EOSEAX NOK
%5ESSMI CHF
%5ESTI SGD


In [20]:
dfacum.head(15)

,activo,divisa,indice
3,UCB.BR,EUR,%5EBFX
9,KBC.BR,EUR,%5EBFX
13,COFB.BR,EUR,%5EBFX
14,ONTEX.BR,EUR,%5EBFX
0,PCAR4.SA,BRL,%5EBVSP
1,ECOR3.SA,BRL,%5EBVSP
2,ENBR3.SA,BRL,%5EBVSP
3,JBSS3.SA,BRL,%5EBVSP
4,RADL3.SA,BRL,%5EBVSP
8,VIVT4.SA,BRL,%5EBVSP


In [21]:
stock_series = dfacum['activo']
index_series = dfacum['indice']

In [23]:
fecha_fin = set_dates('end',ventana)
fecha_inicial = set_dates('start',ventana)

print('la fecha de inicio es',fecha_inicial,' y la de fin es',fecha_fin)

la fecha de inicio es 2020-05-01  y la de fin es 2020-06-30


In [29]:
dfacum = historic_prices_series(stock_series,'1d',fecha_inicial,fecha_fin)

descargando UCB.BR del indice
descargando KBC.BR del indice
descargando COFB.BR del indice
descargando ONTEX.BR del indice
descargando PCAR4.SA del indice
no lo podemos descargar


TypeError: 'NoneType' object is not iterable

Dejamos el Data Frame preparado para el siguiente ejercicio

In [25]:
dfacum.index=dfacum.Date
dfacum=dfacum.loc[:,['stock','Adj Close','High','Low','Open','Volume']]

dfacum.head(10)

AttributeError: 'DataFrame' object has no attribute 'Date'